In [61]:
import pandas as pd

In [162]:
# hawk/dove variable risk with risk attitude adoption and population risk category
#df = pd.read_csv("../hawkdove_variable_2023-10-26T145208_308908.csv")  # 10 runs
#df = pd.read_csv("../hawkdove_variable_2023-10-26T153049_679468.csv")  # 30 runs
#df = pd.read_csv("../hawkdove_variable_2023-10-26T153455_475059.csv")  # 50 runs

df = pd.read_csv("../hawkdove_variable_2023-10-26T154836_183962.csv")  # 100 runs


In [163]:
df

,RunId,iteration,Step,grid_size,risk_adjustment,max_agent_points,percent_hawk,rolling_percent_hawk,population_risk_category,AgentID,risk_level,choice,points
0,0,0,0,10,adopt,24.0,0.47,NaN,13,NaN,NaN,NaN,NaN
1,0,0,1,10,adopt,39.0,0.22,NaN,13,0.0,2.0,dove,15.7
2,0,0,1,10,adopt,39.0,0.22,NaN,13,1.0,5.0,dove,15.7
3,0,0,1,10,adopt,39.0,0.22,NaN,13,2.0,0.0,hawk,9.0
4,0,0,1,10,adopt,39.0,0.22,NaN,13,3.0,8.0,dove,14.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190395,99,99,78,10,adopt,1054.4,0.31,0.519,5,95.0,2.0,hawk,881.7
1190396,99,99,78,10,adopt,1054.4,0.31,0.519,5,96.0,1.0,hawk,791.6
1190397,99,99,78,10,adopt,1054.4,0.31,0.519,5,97.0,2.0,hawk,864.2
1190398,99,99,78,10,adopt,1054.4,0.31,0.519,5,98.0,3.0,hawk,861.3


In [164]:
# limit to model-only fields and drop duplicates, so we can focus on end state for population risk category
model_df = df[['RunId', 'iteration', 'Step', 'percent_hawk', 'rolling_percent_hawk', 'population_risk_category']].drop_duplicates()
model_df

,RunId,iteration,Step,percent_hawk,rolling_percent_hawk,population_risk_category
0,0,0,0,0.47,NaN,13
1,0,0,1,0.22,NaN,13
101,0,0,2,0.64,NaN,13
201,0,0,3,0.53,NaN,13
301,0,0,4,0.32,NaN,13
...,...,...,...,...,...,...
1189900,99,99,74,0.98,0.520333,5
1190000,99,99,75,0.31,0.519333,5
1190100,99,99,76,0.26,0.519667,5
1190200,99,99,77,0.98,0.520000,5


In [165]:
# how long did the simulation run each time?
# batch run was set to step at 200; less than that means convergence logic stopped it early
last_step = model_df.groupby('RunId', as_index=False).agg(last_step=('Step', 'max'))
last_step.tail(10)

,RunId,last_step
90,90,223
91,91,97
92,92,128
93,93,112
94,94,177
95,95,74
96,96,112
97,97,134
98,98,72
99,99,78


In [166]:
# what does the risk distribution look like at the end of each run?

# merge with last step and then filter to just the last step from each run
merged = model_df.merge(last_step, on='RunId')
model_last_step = merged[merged.Step == merged.last_step]
model_last_step.head(10)

,RunId,iteration,Step,percent_hawk,rolling_percent_hawk,population_risk_category,last_step
90,0,0,90,0.51,0.541667,2,90
200,1,1,109,0.22,0.541667,5,109
286,2,2,85,0.13,0.559333,2,85
414,3,3,127,0.14,0.440667,13,127
501,4,4,86,0.90,0.547667,2,86
639,5,5,137,0.95,0.450667,8,137
735,6,6,95,0.81,0.565667,2,95
912,7,7,176,0.91,0.461000,13,176
1043,8,8,130,0.14,0.545667,5,130
1143,9,9,99,0.60,0.527667,5,99


In [167]:
model_last_step.population_risk_category.describe()

count    100.000000
mean       6.000000
std        3.887301
min        2.000000
25%        2.000000
50%        5.000000
75%        9.000000
max       13.000000
Name: population_risk_category, dtype: float64

In [168]:
model_last_step.population_risk_category.unique()

array([ 2,  5, 13,  8,  3,  6,  9, 12, 10,  7])

In [169]:
# graph number of runs in each category

model_grouped = model_last_step.groupby('population_risk_category', as_index=False).agg(runs=('RunId', 'count'))
model_grouped.rename(columns={'population_risk_category': 'risk_category'}, inplace=True)                       
model_grouped

,risk_category,runs
0,2,30
1,3,1
2,5,36
3,6,3
4,7,2
5,8,1
6,9,5
7,10,4
8,12,2
9,13,16


In [170]:
# set labels to group risk categories
def category_label(c):
    if c in [1,2,3,4]:
        return 'majority risk inclined'
    if c in [5,6,7,8]:
        return 'majority risk moderate'
    if c in [9,10,11,12]:
        return 'majority risk avoidant'
    return 'no majority'

model_grouped['category_label'] = model_grouped.risk_category.apply(category_label)
model_grouped

,risk_category,runs,category_label
0,2,30,majority risk inclined
1,3,1,majority risk inclined
2,5,36,majority risk moderate
3,6,3,majority risk moderate
4,7,2,majority risk moderate
5,8,1,majority risk moderate
6,9,5,majority risk avoidant
7,10,4,majority risk avoidant
8,12,2,majority risk avoidant
9,13,16,no majority


In [171]:
import altair as alt
alt.Chart(model_grouped).mark_bar(width=15).encode(
   x=alt.X("risk_category", title="risk category", axis=alt.Axis(tickCount=13),  # 13 categories
           scale=alt.Scale(domain=[1, 13])),
   y=alt.Y("runs", title="Number of runs"),
   color=alt.Color("category_label", title="type")
).properties(title='Distribution of runs by final population risk category')

alt.Chart(...)

In [173]:
# graph some some sample risk distributions within each category, to help understand

combined_chart = None

for category in sorted(model_last_step.population_risk_category.unique()):
    # how many runs ended in this category?
    runs = model_last_step[model_last_step.population_risk_category == category]
    num_runs = len(runs)

    agent_data = []
    # display at max 5 runs for each category
    for run in runs.head(5).itertuples():
       # get agent data from original df for this run and step
       run_agent_df = df[(df.RunId == run.RunId) & (df.Step == run.Step)]
       # group and calculate number of agents per risk level
       grouped = run_agent_df.groupby("risk_level", as_index=False).agg(total=("AgentID", "count"))
       grouped['RunId'] = run.RunId  # set run id for graphing as columned bar chart
       agent_data.append(grouped)

    # combine collected agent data for all runs in this category
    agent_df = pd.concat(agent_data)

    # column bar chart adapted from https://stackoverflow.com/a/71608013
    chart = alt.Chart(agent_df).mark_bar(width=10).encode(
            alt.X('risk_level', title='risk attitude'),
            alt.Y('total'),
            alt.Column('RunId', header=alt.Header(title=f"Category {category} ({num_runs} run{'' if num_runs == 1 else 's'})"))
        ).properties(
             width=200,
             height=200
         )

    # concatenate category charts vertically
    if combined_chart is None:
        combined_chart = chart
    else:
        combined_chart &= chart
            
combined_chart

alt.VConcatChart(...)